# Submit Predictions

This notebook shows how to prepare a submission of your model's predictions on the test data for the computation.

In [1]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))

from pathlib import Path
from natsort import natsorted
import numpy as np
from safetensors.torch import load_model
import torch 
import pandas as pd

from utils import hvatnet
from utils.creating_dataset import LEFT_TO_RIGHT_HAND


## Load pre-trained model

This code loads the pre-trained baseline model - might be different for your model.

In [2]:
device = 'cuda:0'
dtype = torch.float32

# weights = r"C:\Users\feder\Documents\github\BCI_ALVI_challenge\tutorials\logs\test_2_run_fedya\step_3300_loss_0.2750.safetensors"
weights = "/media/lutetia/Extreme SSD/EMG_Yun/BCI_ALVI_challenge/tutorials/logs/test_2_run_fedya/step_6750_loss_0.1370.safetensors"

MODEL_TYPE = 'hvatnet'
model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
                            n_res_blocks=3, n_blocks_per_layer=3,
                            n_filters=128, kernel_size=3,
                            strides=(2, 2, 2), dilation=2, 
                            small_strides = (2, 2))



# model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
#                             n_res_blocks=2, n_blocks_per_layer=5,
#                             n_filters=82, kernel_size=6,
#                             strides=(2, 2, 2), dilation=3, 
#                             small_strides = (2, 2))

model = hvatnet.HVATNetv3(model_config)

load_model(model, weights)
# torch.load(weights, map_location=device)

model = model.to(device).to(dtype)

Number of parameters: 4210788


### Save `submission.cvs` file

This code shows how the data was prepare and downsampled during inference. Make sure that your data is processed similarly!

In [3]:
# DATA_PATH = Path(r"F:\Dropbox (Personal)\BCII\BCI Challenges\2024 ALVI EMG Decoding\dataset_v2_blocks\dataset_v2_blocks")
DATA_PATH = Path("/media/lutetia/Extreme SSD/EMG_Yun/bci-initiative-alvi-hci-challenge/dataset_v2_blocks/dataset_v2_blocks")

test_data_name = 'fedya_tropin_standart_elbow_left'  # shoould match `test_dataset_list` used to train the model


data_folder = DATA_PATH / "amputant" / "left" / test_data_name / "preproc_angles" / "submit"
all_paths = natsorted(data_folder.glob('*.npz'))
print(f'Found {len(all_paths)} samples in {data_folder}')


Found 72 samples in /media/lutetia/Extreme SSD/EMG_Yun/bci-initiative-alvi-hci-challenge/dataset_v2_blocks/dataset_v2_blocks/amputant/left/fedya_tropin_standart_elbow_left/preproc_angles/submit


In [4]:

pred_list = []

# loop over each trial
for i, p in enumerate(all_paths):
    # get EMG data 
    sample = np.load(p)
    myo = sample['data_myo']
    myo = myo[:, LEFT_TO_RIGHT_HAND]

    # predictions will have to be downsampled
    gt_len = myo[::8].shape[0]

    # padding
    target_length = (myo.shape[0] + 255) // 256 * 256
    padded_myo = np.pad(myo, ((0, target_length - myo.shape[0]), (0, 0)), mode='constant', constant_values=0)

    # some prediction. might be slididng window.
    preds = model.inference(padded_myo)
    preds_downsampled = preds[:gt_len]
    print(f"Completed {i+1}/{len(all_paths)}. Loaded data: {myo.shape} - padded to: {padded_myo.shape} - predictions {preds.shape} - downsampled to: {preds_downsampled.shape}")
    pred_list.append(preds_downsampled)

pred_cat = np.concatenate(pred_list, axis=0)
df = pd.DataFrame(pred_cat)
df.head()

Completed 1/72. Loaded data: (3721, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 2/72. Loaded data: (3725, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 3/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 4/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 5/72. Loaded data: (3729, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (467, 20)
Completed 6/72. Loaded data: (3726, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 7/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 8/72. Loaded data: (3728, 8) - padded to: (3840, 8) - predictions (480, 20) - downsampled to: (466, 20)
Completed 9/72. Loaded data: (3724, 8) - padded to: (3840, 8) - predictions (480, 20) - 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.190404,-0.058046,0.461790,0.109999,0.206227,-0.006641,0.475586,0.088680,0.192175,0.017804,0.601238,0.329655,0.228485,0.013447,0.434309,0.167887,0.072154,-0.039544,0.389007,0.002870
1,0.203336,-0.046114,0.452863,0.119911,0.204110,-0.012413,0.463128,0.096496,0.191725,0.011688,0.603316,0.329274,0.238322,-0.000297,0.423337,0.162530,0.079310,-0.031150,0.375805,0.001825
2,0.178536,-0.040807,0.469255,0.148645,0.190936,-0.012788,0.429239,0.090321,0.190977,0.013575,0.579209,0.319150,0.223728,0.003910,0.429668,0.179140,0.104281,-0.054424,0.363032,0.012655
3,0.162663,-0.034840,0.499209,0.180319,0.181307,-0.011435,0.472396,0.124288,0.172313,0.009265,0.619048,0.347650,0.224190,0.001809,0.457648,0.213931,0.081826,-0.035435,0.391104,0.056601
4,0.170310,-0.044011,0.492226,0.164777,0.185370,-0.011827,0.461971,0.112577,0.173891,0.006542,0.627049,0.356788,0.218134,0.014589,0.474839,0.221507,0.080414,-0.031257,0.412126,0.062509


In addition to the predictions, your data should also include a sample id column.

In [5]:
df.insert(0, "sample_id", range(1, 1 + len(df)))

Finally, save to a CSV file. This is what you'll upload to Kaggle for the competition.

In [6]:
df.to_csv('submit_file_Wave_MSE_moving_average10.csv', index=False)

In [7]:
df.shape

(26829, 21)